In [ ]:
!cd Split/
!ls

In [ ]:
!python splitter.py -f inputVideo.avi -s 1 -v libx264

In [ ]:
!mkdir Videos
!mv [inputVideo]* Videos

In [ ]:
!cd Videos
!sudo rm Videos/inputVideo.avi
import os
os.chdir("Videos")

In [ ]:

myArray = os.listdir()
myArray

In [ ]:
myArray.sort()
spareArray = myArray

In [ ]:
myArray = ["Videos/" + s for s in myArray]

In [ ]:
!pip install -q "tensorflow>=1.7" "tensorflow-hub" "imageio"

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub


tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import random
import re
import os
import shutil
import tempfile
import cv2
import numpy as np

import imageio
from IPython import display

from urllib import request
import urllib.request

In [ ]:
UCF_ROOT = "http://crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
_CACHE_DIR = tempfile.mkdtemp()

def list_ucf_videos():
    global _VIDEO_LIST
    if not _VIDEO_LIST:
      index = request.urlopen(UCF_ROOT).read().decode("utf-8")
      videos = re.findall("(v_[\w_]+\.avi)", index)
      _VIDEO_LIST = sorted(set(videos))
    return list(_VIDEO_LIST)

def fetch_ucf_video(video):
    cache_path = os.path.join(_CACHE_DIR, video)
    if not os.path.exists(cache_path):
      urlpath = request.urljoin(UCF_ROOT, video)
      data = request.urlopen(urlpath).read()
      open(cache_path, "wb").write(data)
    return cache_path

def crop_center_square(frame):
    x, y = frame.shape[0:2]
    min_dim = min(x, y)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_x:start_x+min_dim, start_y:start_y+min_dim]

def load_video(path, max_frames=0, resize=(224, 224)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
      while True:
        ret, frame = cap.read()
        if not ret:
          break
        frame = crop_center_square(frame)
        frame = cv2.resize(frame, resize)
        frame = frame[:, :, [2, 1, 0]]
        frames.append(frame)

        if len(frames) == max_frames:
          break
    finally:
      cap.release()
    return np.array(frames) / 255.0

def animate(images):
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave('./animation.gif', converted_images, fps=25)
    with open('./animation.gif','rb') as f:
        display.display(display.Image(data=f.read(), height=300))
        
def substring_after(subString, listOfWord):
    return listOfWord.partition(subString)[2]

In [ ]:
KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map_600.txt"
with request.urlopen(KINETICS_URL) as obj:
    labels = [line.decode("utf-8").strip() for line in obj.readlines()]
print("Found %d labels." % len(labels))

In [ ]:
nameOfSavedVideos = []

In [ ]:
for video_name, spare_name in zip(myArray, spareArray):
    os.chdir("/home/jupyter/Split/")
    if video_name not in nameOfSavedVideos:
        sample_video = load_video(video_name)
        model_input = np.expand_dims(sample_video, axis=0)

        with tf.Graph().as_default():
            i3d = hub.Module("https://tfhub.dev/deepmind/i3d-kinetics-600/1")
            input_placeholder = tf.compat.v1.placeholder(shape=(None, None, 224, 224, 3), dtype=tf.float32)
            logits = i3d(input_placeholder)
            probabilities = tf.nn.softmax(logits)
            with tf.compat.v1.train.MonitoredSession() as session:
              [ps] = session.run(probabilities, feed_dict={input_placeholder: model_input})

        for i in np.argsort(ps)[::-1][:1]:
            print("%-22s %.2f%%" % (labels[i], ps[i] * 100))
            #Create folder
            mypath = "/home/jupyter/Split/Videos/"
            folder_name_for_test_videos = (labels[i])
            if not os.path.isdir(mypath + folder_name_for_test_videos):
               os.makedirs(mypath + folder_name_for_test_videos)
            #Move related videos to related files
            os.rename(mypath + spare_name, mypath + folder_name_for_test_videos + "/" + spare_name)
            
            os.chdir("/home/jupyter/Split/Videos/" + folder_name_for_test_videos)
            
            !ffmpeg -i $spare_name -qscale:v 2 thumb%04d.jpg
                        
            listOfEachImageFileByFrame = os.listdir()
            myFramenewlist = [x for x in listOfEachImageFileByFrame if x.startswith("thumb")]
            myFramenewlist.sort()

            font = cv2.FONT_HERSHEY_SIMPLEX

            firstImg = cv2.imread(myFramenewlist[0], cv2.IMREAD_COLOR)
            height, width, channels = firstImg.shape

            for image in myFramenewlist:
                img = cv2.imread(image, cv2.IMREAD_COLOR)
                cv2.putText(img, "%-22s: %.2f%%" % (labels[i], ps[i] * 100) , (25,25), font, 1, (255,0,0), 1, cv2.LINE_AA)
                cv2.imwrite(image, img)
            
            !sudo rm $spare_name
            !ffmpeg -framerate 30 -i thumb%04d.jpg $spare_name
            !sudo rm thumb*

In [ ]:
os.chdir("/home/jupyter/Split/Videos")
listOfVideoTypes = os.listdir()

for videoFolder in listOfVideoTypes:
    os.chdir("/home/jupyter/Split/Videos/" + videoFolder)
    
    listOfEachVideoFile = os.listdir()
    mynewlist = [x for x in listOfEachVideoFile if x.startswith("inputVideo")]
    mynewlist.sort()

    counter = 0

    while bool(mynewlist) is True:
        yFile = "inputVideo-0"
        f = open('helloworld.txt','w')

        for xFile in mynewlist:
            if xFile == mynewlist[0]:
                f.write("file" + " '" + xFile + "'" + "\n")        

            if substring_after("inputVideo-", xFile).startswith(str(int(substring_after("inputVideo-", yFile)[0]) + 1)):
                f.write("file" + " '" + xFile + "'" + "\n")

            yFile = xFile

        f.close()

        nameOfOutputVideos = "output" + str(counter) + ".avi"

        !ffmpeg -f concat -i helloworld.txt -c copy $nameOfOutputVideos
        !xargs rm < helloworld.txt
        !sudo rm helloworld.txt

        listOfEachVideoFile = os.listdir()
        mynewlist = [x for x in listOfEachVideoFile if x.startswith("inputVideo")]
        mynewlist.sort()

        counter = counter + 1

In [ ]:
os.chdir("/home/jupyter/Split/Videos/")
!find . -mindepth 2 -type f -print -exec mv {} . \;

!rename 's/^inputVideo-//' *
!rename 's/.avi//' *

listOfEachVideoFile = os.listdir()
mynewlist = [x for x in listOfEachVideoFile if x[0].isdigit()]
mynewlist = [int(x) for x in mynewlist]
mynewlist.sort()

f = open('helloworld.txt','w')

for xFile in mynewlist:
    f.write("file" + " '" + str(xFile) + "'" + "\n")        

f.close()

In [ ]:
nameOfOutputVideos = "output.avi"
!ffmpeg -f concat -i helloworld.txt -c copy $nameOfOutputVideos

In [ ]:
!xargs rm < helloworld.txt
!sudo rm helloworld.txt